# Understanding Aqua's Operator Flow

_donny@, 14-May-20_

**Table of Contents**

1  [Overview](#Overview) <br>
&emsp;&emsp;1.1  [🚨🚨🚨Disclaimers🚨🚨🚨](#🚨🚨🚨Disclaimers🚨🚨🚨) <br>
&emsp;&emsp;1.2  [Basic Design Principles](#Basic-Design-Principles) <br>
&emsp;&emsp;1.3  [Basic Definitions](#Basic-Definitions) <br>
&emsp;&emsp;1.4  [Object Structure](#Object-Structure) <br>
&emsp;&emsp;1.5  [Motivating Example](#Motivating-Example) <br>
2  [State Functions and Measurements](#State-Functions-and-Measurements) <br>
&emsp;&emsp;2.1  [.eval(state) vs .compose(state)](#.eval(state)-vs-.compose(state)) <br>
&emsp;&emsp;2.2  [Arithmetic Operations](#Arithmetic-Operations) <br>
3  [PrimitiveOps](#PrimitiveOps) <br>
&emsp;&emsp;3.1  [OperatorStateFn](#OperatorStateFn) <br>
4  [ListOps](#ListOps) <br>
&emsp;&emsp;4.1  [Hierarchical ListOps](#Hierarchical-ListOps) <br>
5  [Converters](#Converters) <br>
&emsp;&emsp;5.1  [Evolutions, exp_i() and the EvolvedOp](#Evolutions,-exp_i()-and-the-EvolvedOp) <br>
&emsp;&emsp;5.2  [Expectations](#Expectations) <br>
&emsp;&emsp;5.3  [Executing CircuitStateFns with the CircuitSampler](#Executing-CircuitStateFns-with-the-CircuitSampler) <br>
6  [Conclusion](#Conclusion) <br>

## Overview

A library for Quantum Algorithms & Applications is more than a collection of procedural code wrapped in Python functions. It needs to provide tools to make writing algorithms simple and easy. This is the layer of modules between the circuits and algorithms, providing the language and computational primitives for Quantum algorithms research.

In Aqua, we call this layer the Operator flow. It works by unifying computation with theory through the common language of functions and operators in a way which preserves physical intuition and programming freedom. In the Operator flow, we construct functions over binary variables, manipulate those functions with operators, and evaluate properties of these functions with measurements. 

Below, we'll describe the key players in the Operator flow, and show some examples of how they can be used to construct some familiar quantum algorithms. This notebook assumes a basic familiarity with Quantum Algorithms.

### 🚨🚨🚨Disclaimers🚨🚨🚨
1. The Operator flow is a large and self-referential system. Do not worry if you don't immediately feel comfortable with all of its moving parts - like any system of abstractions, it can take time and play to feel comfortable. We feel that this is a worthwhile investment, as once you begin to feel comfortable with the system, algorithm development is super fast and easy.

2. The Qiskit 0.19 release includes the ***v0*** version of the Operator flow, and there are plenty of kinks to work out. We appreciate your patience, diligence to point out bugs, feedback (by [filing bugs in github](https://github.com/Qiskit/qiskit-aqua/issues) ), and contributions as the Operator flow matures.

### Basic Design Principles

* *Batteries Included* - We prioritize quick and easy access to the tools you're most likely to reach for when prototyping.

* *Physically Formal* - The Operator flow was built to be mathematically formal down to its very bottom primitives, so that it may serve as a lingua franca between the theory and implementation of Quantum algorithms.

* *Powerful OR Readable* - We've tried to let any operation be as succinct or verbose as you want it to be, including both rich syntactic sugar and long-form interfaces.

* *Fast by Default* - We use thoughtfully selected defaults so that algorithms run as fast as possible at many scales, and so you only need to toggle the settings you care about. We want you to be able to trust that we're running your problem as fast as we can, and if you find a way to run it faster, you should [let us know!](https://github.com/Qiskit/qiskit-aqua/issues)

### Basic Definitions

Before getting into the details of the code, it's important to note that three mathematical concepts unpin the Operator flow. We derive most of the inspiration for the code structure from [John Watrous's Quantum Information formalism](https://cs.uwaterloo.ca/~watrous/TQI/) (but do not follow it exactly), so it may be worthwhile to review Chapters I and II, which are free online, if you feel the concepts are not clicking.

1. **"State functions"** - An $n$-qubit State function is a complex function over $n$ binary variables, which we will often refer to as _$n$-qubit binary strings_. For example, the traditional quantum "zero state" is a 1-qubit state function, with a definition of $Zero(0) = 1$ and $Zero(1) = 0$. It is often convenient to think about State functions as traditional kets (i.e. $Zero = |0\rangle$), but be warned, **State functions need not be normalized.** They are just functions.

1. **"Operators"** - An $n$-qubit Operator is a linear function taking $n$-qubit state functions to $n$-qubit state functions. For example, the Pauli X Operator is defined by $X(Zero) = One$ and $X(One) = Zero$. Therefore, $X(Zero)(0) = 0$. Equivalently, an Operator can be defined as a complex function over two n-qubit binary strings, and it is sometimes convenient to picture things this way. By this definition, our Pauli $X$ can be defined by its typical matrix elements, $X(0, 0) = 0$, $X(1, 0) = 1$, $X(0, 1) = 1$, $X(1, 1) = 0$. In the Operator flow, our Operators are all binary, square Operators (for now).

1. **"Measurements"** - An $n$-qubit Measurement is a functional taking n-qubit State functions to complex values. For example, a Pauli Z Measurement can be defined by $Z_{meas}(Zero) = 1$ and $Z_{meas}(One) = -1$. Measurements are simply adjoints of State Functions, and can be imagined by quantum mechanics' _bra_ notation (i.e. $Zero_{meas} = Zero^{\dagger} = \langle 0|$). In thinking of Measurements as adjoints of State functions, note that $Z_{meas}$ above is equivalent to $1 - 2*(\langle Zero| + \langle One|)$.

**Throughout this tutorial, we will refer to the mathematical function (of type 1-3 above) that a Python Object represents as that Object's *characteristic function*.**

#### Chaining vs. Composition
Note that a chained expression of the functions above can be written in two ways:
* As function chaining: $Z(X(Zero))(0) = 0$
* As function composition: $Zero_{meas} \circ Z \circ X \circ Zero = 0$

In the Operator flow, chaining is used via the `.eval()` method, while composition is used via `.compose()` or `@` (more on this below).

### Object Structure

The Operator flow includes two primary groups of actors: 
* Operators, objects which represent functions or functionals, all deriving from `OperatorBase`: 
    * `PrimitiveOp`s - Basic Operator building blocks, whose behavior are defined by some internal computational primitive from Terra, such as a Terra's `Pauli` (`PauliOp`), `Operator` (`MatrixOp`), or `QuantumCircuit` (`CircuitOp`).
    * `StateFn`s - A class for representing both State functions and Measurements, the behavior of which is defined by some internal primitive, e.g. a simple dict (`DictStateFn`), Terra's `Statevector` (`VectorStateFn`), a `QuantumCircuit` (`CircuitStateFn`), or an OperatorBase representing a density operator (`OperatorStateFn`).
    * `ListOp`s - Classes for representing composite Operators, State functions, and Measurements constructed from expressions of others, such as addition (`SummedOp`), composition (`ComposedOp`), tensor product (`TensoredOp`), and concatination into a list (`ListOp`).
    * `EvolvedOp` - A special case of a `PrimitiveOp` holding an `Operatorbase` as its primitive, serving as a placeholder for an evolution algorithm to convert later into an Operator approximating the exponentiation of the `OperatorBase`.
* Converters, objects which manipulate Operators, all deriving from `ConverterBase`:
    * `Expectation`s - Traverse over an Operator and replace OperatorStateFn measurements into Z-basis or matrix measurements approximating the expectation value of the measurement.
    * `Evolution`s - Traverse over an Operator and replace `EvolvedOp`s with circuits approximating the evolution.
    * `CircuitSampler` - Traverse over an Operator and replace `CircuitStateFn`s with `DictStateFn`s approximating them, perhaps sampled from a quantum device.
    * Other converters, e.g. `AbelianGrouper`, `PauliBasisChange`
    
Don't worry if this seems like a whirlwind. We'll discuss each of these in detail in the next section, and you can also find more detail in the [Operator flow API reference](https://qiskit.org/documentation/apidoc/qiskit.aqua.operators.html#module-qiskit.aqua.operators).

### Motivating Example

Below, we'll show some basic code to highlight the power of the Operator flow. The code is simple - we'll evolve a Bell state by an H2 Hamiltonian using a simple Trotter expansion and measure the energy expectation value of the Hamiltonian at various evolution times. You are obviously not expected to understand it, but inspect it to gain some context and see how it simplifies otherwise complex constructions. Also note that this is a slightly verbose form of this flow for instruction's sake.

In the Converters section of this notebook, we'll walk through the example in more detail.

In [1]:
from qiskit import BasicAer
from qiskit.circuit import Parameter
from qiskit.aqua.operators import *
import numpy as np

two_qubit_H2 =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)

bell = CX @ (Plus^Zero)
evo_time = Parameter('θ')
evolution_op = (evo_time*two_qubit_H2).exp_i()
h2_measurement = StateFn(two_qubit_H2).adjoint()
evo_and_meas = h2_measurement @ evolution_op @ bell

trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki()).convert(evo_and_meas)
diagonalized_meas_op = PauliExpectation().convert(trotterized_op)
evo_time_points = list(range(8))
h2_trotter_expectations = diagonalized_meas_op.bind_parameters({evo_time: evo_time_points})

# Energies by Trotter expansion and sampled measurement
sampler = CircuitSampler(backend=BasicAer.get_backend('qasm_simulator'))
sampled_trotter_exp_op = sampler.convert(h2_trotter_expectations)
sampled_trotter_energies = sampled_trotter_exp_op.eval()
print('Sampled Trotterized energies:\n {}'.format(np.real(sampled_trotter_energies)))

Sampled Trotterized energies:
 [-0.6474109  -0.63954839 -0.66557691 -0.65210217 -0.64456728 -0.7052324
 -0.8078819  -0.92018269]


## State Functions and Measurements

The most basic unit of computation in the Operator flow is the state function, abbreviated as `StateFn`, a simple complex function over binary variables. There are several `StateFn` instances built in for convenience, `Zero, One, Plus, Minus`, and four ways a `StateFn` can be defined, `DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn`.

Every `StateFn` has three properties:
* `primitive` - The data structure defining the underlying function's behavior. For `DictStateFn`, this object is a `dict`, etc.
* `coeff` - A coefficient multiplying the characteristic function, i.e. `StateFn(my_primitive, coeff=3) == StateFn(my_primitive) * 3`. Note that `coeff` can be int, float, complex or a free `Parameter` object (from `qiskit.circuit` in Terra) to be bound later using `my_op.bind_parameters`.
* `is_measurement` - Whether this `StateFn` is a state function or a measurement (remember, a measurement is just the adjoint of a state function).

In [2]:
from qiskit.aqua.operators import (StateFn, Zero, One, Plus, Minus, 
                                   DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn)

In [3]:
print(Zero)
print(Zero.adjoint())

DictStateFn({'0': 1})
DictMeasurement({'0': 1})


The function this object represents, mapping a binary string to a complex value, can be accessed via the `.eval` method.

In [4]:
print(Zero.eval('0'))
print(Zero.eval('1'))
print(One.eval('1'))
print(Plus.eval('0'))
print(Minus.eval('1'))

1.0
0.0
1.0
(0.70710678118655+0j)
(-0.70710678118655+0j)


This should look familiar as something analogous to the quantum state or wavefunctions you know, $|0\rangle$, $|1\rangle$, $|+\rangle$, and $|-\rangle$, but with a key difference: these statefunctions need not be normalized. `statefunction1 + statefunction2` behaves exactly as you'd expect mathematical functions to behave, producing `statefunction3`, where `statefunction3(x) = statefunction1(x) + statefunction2(x)`.

In [5]:
print((One + One + One).eval('1'))
print((One + One + One))
print((Plus + Minus).eval('0'))
print((Plus + Minus).eval('1'))

3.0
DictStateFn({'1': 1}) * 3.0
(1.4142135623731+0j)
0j


The behavior of each StateFn type is defined internally by some data structure, which we call the primitive, and a complex coefficient. The `DictStateFn` is the simplest type, holding a dict primitive of {string: complex} value pairs. `Zero` and `One` are simply these. Any value missing from the dict is simply equal to 0.

In [6]:
print(Zero)
print(One)
print(Zero.primitive)
print(Zero.coeff)
print((Zero + Zero + Zero).coeff)
print((Zero * 2j).coeff)

DictStateFn({'0': 1})
DictStateFn({'1': 1})
{'0': 1}
1.0
3.0
2j


For simplicity, the `StateFn` class also doubles as the class for Measurement. The adjoint of a `StateFn` is simply a measurement defined by the same primitive as the state. This is conceptually identical to the idea that a bra is the adjoint of a ket, and it is convenient to think of it that way.

Just as the State function over binary variables is accessible via `.eval`, so to the Measurement functional over State functions is available via `.eval`. As we'll see below, the availability of the underlying function through `.eval` is also true for the Operators.

In [7]:
print(One.adjoint())
print(Zero.adjoint().eval(One))
print(Zero.adjoint().eval(Zero))
print(Zero.adjoint().eval(Plus))

DictMeasurement({'1': 1})
0.0
1.0
(0.70710678118655+0j)


Note that we can also perform function composition between a State function and measure using `.compose`, and call `.eval()` with no argument later.

In [8]:
print(Zero.adjoint().compose(One).eval())
print(Zero.adjoint().compose(Zero).eval())
print(Zero.adjoint().compose(Plus).eval())

0.0
1.0
(0.70710678118655+0j)


### `.eval(state)` vs `.compose(state)`

Although they look similar, there is a critical difference between `eval` and `compose`. `eval` will do everything in its power to complete the evaluation, including totally unscalable computation, such as conversion to exponential-sized matrices. Compose will not perform any unscalable computation under the hood, and will simply represent the composition symbolically (returning a `ComposedOp`, see `ListOps` below) if it does not know how to complete the composition efficiently.

`eval(state)` can be seen as a convenience for hacking, while `compose` the more typical route for actual usage within quantum algorithms. In some sense, `eval()` will always try to complete the computation its told to complete, and if the computation isn't scalable, it's up you to take more complex steps to make it faster (e.g. run it on a quantum computer).

### Arithmetic Operations

A rich set of arithmetic operations between StateFns (and all `OperatorBase`s in fact) are supported, including:

|Overload|Operation                                                     |
|:----:|:---------------------------------------------------------------|
| +  | addition                                                      |
| -  | subtraction, negation (scalar multiplication by               |
| *  | scalar multiplication                                         |
| /  | scalar division                                               |
| @  | composition                                                   |
| ^  | tensor product or tensor power (tensor with self n times)     |
| ** | composition power (compose with self n times)                 |
| == | equality                                                      |
| ~  | adjoint, alternating between a State Function and Measurement |

This arithmetic, along with the Operators, allows quick and easy construction of many states for Quantum Algorithms. However, **one must be careful to use parentheses properly when performing arithmetic in this way.**

In [9]:
print(600 * ((One^5) + (Zero^5)))
print(One^Zero^3)

DictStateFn({'11111': 1.0, '00000': 1.0}) * 600.0
DictStateFn({'101010': 1})


States can also be easily converted between primitives. Below we'll see that we can also represent State functions by vectors and Quantum circuits. Note that to be consistent with the Operators, we use `to_matrix_op` to convert to a `VectorStateFn`.

In [50]:
print(((Plus^Minus)^2).to_matrix_op())
# TODO round
print(((Plus^One)^2).to_circuit_op())
print(((Plus^One)^2).to_matrix_op().sample())

VectorStateFn(Statevector([ 0.25-0.j, -0.25+0.j,  0.25-0.j, -0.25+0.j, -0.25+0.j,
              0.25-0.j, -0.25+0.j,  0.25-0.j,  0.25-0.j, -0.25+0.j,
              0.25-0.j, -0.25+0.j, -0.25+0.j,  0.25-0.j, -0.25+0.j,
              0.25-0.j],
            dims=(2, 2, 2, 2)))
CircuitStateFn(
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ X ├
     ├───┤
q_3: ┤ H ├
     └───┘
)
{'1101': 0.259765625, '0111': 0.251953125, '1111': 0.2490234375, '0101': 0.2392578125}


In fact, `Plus` and `Minus` are `CircuitStateFn`s.

In [11]:
print(Plus)
print(Minus)

CircuitStateFn(
     ┌───┐
q_0: ┤ H ├
     └───┘
)
CircuitStateFn(
     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     └───┘└───┘
)


Constructing a `StateFn` is easy. The `StateFn` class also serves as a factory, and can take any applicable primitive in its constructor and return the correct `StateFn` subclass. Right now the following primitives can be passed into the constructor, listed alongside the `StateFn` subclass they produce:

* `str` (equal to some basis bitstring) → `DictStateFn`
* `dict` → `DictStateFn`
* Qiskit `Result` object → `DictStateFn`
* `list` → `VectorStateFn`
* `np.ndarray` → `VectorStateFn`
* Terra's `quantum_info.Statevector` → `VectorStateFn`
* `QuantumCircuit` → `CircuitStateFn`
* `Instruction` → `CircuitStateFn`
* `OperatorBase` → `OperatorStateFn`

In [12]:
print(StateFn({'0':1}))
print(StateFn({'0':1}) == Zero)

print(StateFn([0,1,1,0]))

from qiskit.circuit.library import RealAmplitudes
print(StateFn(RealAmplitudes(2)))


DictStateFn({'0': 1})
True
VectorStateFn(Statevector([0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j],
            dims=(2, 2)))
CircuitStateFn(
     ┌──────────┐     ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ RY(θ[0]) ├──■──┤ RY(θ[2]) ├──■──┤ RY(θ[4]) ├──■──┤ RY(θ[6]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ RY(θ[1]) ├┤ X ├┤ RY(θ[3]) ├┤ X ├┤ RY(θ[5]) ├┤ X ├┤ RY(θ[7]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘└───┘└──────────┘
)


## PrimitiveOps

Operators are functions which take StateFns to StateFns, and `PrimitiveOp`s are the building blocks for these Objects. For example, the identity Operator takes any StateFn to itself, and we can build this Operator efficiently out of `PauliOps`. Just like State Functions and Measurements, this characteristic function of the Operator is accessible via the `.eval` method, and the behavior of the function is defined by a computational primitive. If you pass a bitstring into an Operator's `eval` function, it will simply treat it as the corresponding basis StateFn.

Every `PrimitiveOp` has two properties:
* `primitive` - The data structure defining the underlying function's behavior. For `DictStateFn`, this object is a `dict`, etc.
* `coeff` - A coefficient multiplying the characteristic function. Note that `coeff` can be int, float, complex or a free `Parameter` object (from `qiskit.circuit` in Terra) to be bound later using `my_op.bind_parameters`.

Just like StateFn, `PrimitiveOp` is also a factory for creating the correct type of `PrimitiveOp` for a given primitive. Right now the following primitives can be passed into the constructor, listed alongside the `PrimitiveOp` subclass they produce:

* Terra's `Pauli` → `PauliOp`
* `Instruction` → `CircuitOp`
* `QuantumCircuit` → `CircuitOp`
* 2d `list` → `MatrixOp`
* `np.ndarray` → `MatrixOp`
* `scipy.spmatrix` → `MatrixOp`
* Terra's `quantum_info.Operator` → `MatrixOp`

In [13]:
from qiskit.aqua.operators import X, Y, Z, I, CX, T, H, S, PrimitiveOp

In [14]:
print(X)
print(X.eval(One))
print(X.eval(One) == Zero)
print(3*X.eval(One))

print(CX)
print(CX.eval('01'))
print(CX.eval('01').eval('11'))
# The same expression, written in compositions:
print(((~One^2) @ CX @ (Zero^One)).eval())

# We can build complicated Operators very easily
print(((H^5) @ ((CX^2)^I) @ (I^(CX^2)))**2)

X
DictStateFn({'0': (1+0j)})
True
DictStateFn({'0': (1+0j)}) * 3.0
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
            dims=(2, 2)))
(1+0j)
(1+0j)
          ┌───┐          ┌───┐     
q_0: ──■──┤ H ├───────■──┤ H ├─────
     ┌─┴─┐└───┘┌───┐┌─┴─┐└───┘┌───┐
q_1: ┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     └───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_2: ──■──┤ X ├┤ H ├──■──┤ X ├┤ H ├
     ┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_3: ┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     └───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_4: ─────┤ X ├┤ H ├─────┤ X ├┤ H ├
          └───┘└───┘     └───┘└───┘


**Some important notes:**
* Qiskit prints circuits in small-endianness, meaning that qubit 0 in the Operator construction will print at the **bottom** of the circuit. 
* Circuits grow left to right, with the state beginning to the left, whereas mathematical notation grows right to left, with the starting state to the right. For this reason, circuits will appear to print in reverse from how you've constructed them.
* We can convert a `CircuitOp` into `CircuitStateFn` simply by composing `Zero` or another `CircuitStateFn`

In [15]:
print(((H^I^I)@(X^I^I)))

# Note that this is now printed as a `CircuitStateFn`!
print((((H^5) @ ((CX^2)^I) @ (I^(CX^2)))**2) @ (Minus^5))

               
q_0: ──────────
               
q_1: ──────────
     ┌───┐┌───┐
q_2: ┤ X ├┤ H ├
     └───┘└───┘
CircuitStateFn(
     ┌───┐┌───┐     ┌───┐          ┌───┐     
q_0: ┤ X ├┤ H ├──■──┤ H ├───────■──┤ H ├─────
     ├───┤├───┤┌─┴─┐└───┘┌───┐┌─┴─┐└───┘┌───┐
q_1: ┤ X ├┤ H ├┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     ├───┤├───┤└───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_2: ┤ X ├┤ H ├──■──┤ X ├┤ H ├──■──┤ X ├┤ H ├
     ├───┤├───┤┌─┴─┐└───┘├───┤┌─┴─┐└───┘├───┤
q_3: ┤ X ├┤ H ├┤ X ├──■──┤ H ├┤ X ├──■──┤ H ├
     ├───┤├───┤└───┘┌─┴─┐├───┤└───┘┌─┴─┐├───┤
q_4: ┤ X ├┤ H ├─────┤ X ├┤ H ├─────┤ X ├┤ H ├
     └───┘└───┘     └───┘└───┘     └───┘└───┘
)


`PrimitiveOp`s enjoy the same rich set of arithmetic operations and conversion options as the `StateFn`s.

In [16]:
circ_op = (H^I) @ CX @ (H^I)
mat_op = circ_op.to_matrix_op()
print(mat_op)
print(mat_op.to_circuit_op()) # attempt to convert back to circuit, will use a `UnitaryGate`
print(circ_op.to_pauli_op()) # convert to sum of Paulis, more on `SummedOp`s below

Operator([[ 1.-0.j,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  1.-0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  1.-0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
     ┌──────────┐
q_0: ┤0         ├
     │  unitary │
q_1: ┤1         ├
     └──────────┘
SummedOp(
[0.5 * II,
0.5 * IZ,
0.5 * ZI,
-0.5 * ZZ])


### `OperatorStateFn`

There is a special type of `StateFn` which we did not discuss above, called an `OperatorStateFn`, whose behavior is defined by an `OperatorBase` density operator. As a measurement, this is equivalent to an expectation measurement by an `OperatorBase` observable. Note that measurement case is by far the more common usage of this Class.

In [17]:
print(StateFn(Z))
print(~StateFn(Z))

print(StateFn(Z).adjoint().eval(Zero))
print(StateFn(Z).adjoint().eval(Plus))

# Note that Pauli measurements of `DictStateFns` are done by bitwise operations under the hood,
# so they can be huge. Here's a 400-qubit measurement:
print(StateFn(Z^400).adjoint().eval((One^Zero)^200))

# We can construct expectation values this way, but can't always perform them efficiently. 
# See `Expectations` below.
two_qubit_H2 =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)
print(StateFn(two_qubit_H2).adjoint().eval(Zero^Zero))

OperatorStateFn(Z)
OperatorMeasurement(Z)
(1+0j)
0j
(1+0j)
(-1.06365328+0j)


## ListOps

If you've already played around with the above, you'll notice that you can easily perform operations between `OperatorBase`s which we may not know how to perform efficiently in general (or simply haven't implemented an efficient procedure for yet), such as addition between `CircuitOp`s. In those cases, you may receive a `ListOp` result (or subclass thereof) from your operation representing the lazy execution of the operation. For example, if you attempt to add together a `DictStateFn` and a `CircuitStateFn`, you'll receive a `SummedOp` representing the sum of the two. This composite State function still has a working `eval` (but may need to perform a non-scalable computation under the hood, such as converting both to vectors).

These composite `OperatorBase`s are how we construct increasingly complex and rich computation out of `PrimitiveOp` and `StateFn` building blocks.

Every `ListOp` has four properties:
* `oplist` - The list of `OperatorBase`s which define the behavior of the `ListOp`'s characteristic function.
* `combo_fn` - The function taking a list of classical scalars to an output value which defines how to combine the outputs of the `oplist` items' functions to produce the `ListOp`'s characteristic function. For broadcasting simplicity, this function is defined over NumPy arrays.
* `coeff` - A coefficient multiplying the characteristic function. Note that `coeff` can be int, float, complex or a free `Parameter` object (from `qiskit.circuit` in Terra) to be bound later using `my_op.bind_parameters`.
* `abelian` - Indicates whether the Operators in `oplist` are known to mutually commute (usually set after being converted by the `AbelianGrouper` converter).

Note that `ListOp` supports typical iteration overloads, so you can use indexing like `my_op[4]` to access the `OperatorBase`s in `oplist`.

In [53]:
print(Zero + Plus)
print((Zero + Plus)[0])
print((Zero + Plus).eval('0'))

SummedOp(
[DictStateFn({'0': 1}),
CircuitStateFn(
     ┌───┐
q_0: ┤ H ├
     └───┘
)])
DictStateFn({'0': 1})
(1.7071067811865501+0j)


Or for composition, you might receive a `ComposedOp`. Note that just as in addition, we can sometimes perform the composition efficiently, such as between two circuit State functions. However, when composing a Measurement with a State function, we will always receive a `ComposedOp` back to reflect the fact that the State Function has been measured.

In [19]:
print('Dict measurement with Circuit statefn:')
print(~One @ Minus)
print((~One @ Minus).eval())
print('\nCircuit measurement with Circuit statefn:')
print(~One.to_circuit_op() @ Minus)
print('\nDict measurement with vector statefn:')
print(~One @ Minus.to_matrix_op())

Dict measurement with Circuit statefn:
ComposedOp(
[DictMeasurement({'1': 1}),
CircuitStateFn(
     ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     └───┘└───┘
)])
(-0.70710678118655-0j)

Circuit measurement with Circuit statefn:
ComposedOp(
[CircuitMeasurement(
     ┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘
),
DictStateFn({'0': 1})])

Dict measurement with vector statefn:
ComposedOp(
[DictMeasurement({'1': 1}),
VectorStateFn(Statevector([ 0.70710678-0.j, -0.70710678+0.j],
            dims=(2,)))])


### Hierarchical ListOps

The base `ListOp` class is a special type of ListOp. Its `combo_fn` is the identity - it accepts a list and produces that same list. This allows the ListOp's characteristic function to represent the vectorization of the `oplist`'s characteristic functions into a list, and the construction of large and interesting Operators.

In [20]:
print((~StateFn(Z) @ ListOp([One, Zero, Plus, Minus])).eval())
print(np.asarray((~StateFn(ListOp([Z, X])) @ ListOp([One, Zero, Plus, Minus])).eval()))

[(-1+0j), (1+0j), 0j, 0j]
[[-1.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j -1.+0.j]]


For example, building a Quantum Kernel is trivial using `ListOp`s.

In [21]:
kernel_states = ListOp([One, Zero, Plus, Minus])
print(np.asarray((~kernel_states @ kernel_states).eval()))

[[ 1.        +0.j  0.        +0.j  0.70710678+0.j -0.70710678-0.j]
 [ 0.        +0.j  1.        +0.j  0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j  0.70710678+0.j  1.        +0.j  0.        +0.j]
 [-0.70710678+0.j  0.70710678+0.j  0.        +0.j  1.        +0.j]]


Note that in contrast with Terra, binding a list of values to a parameter in `bind_parameters` will return a `ListOp` of copies of the Operator bound with each parameterization.

In [60]:
from qiskit.circuit.library import ZZFeatureMap
feat_map = PrimitiveOp(ZZFeatureMap(2)) @ Zero
mapped_features = feat_map.bind_parameters(dict(zip(feat_map.primitive.ordered_parameters, np.random.random((2,2)).tolist())))
qkernel_mat = (~mapped_features @ mapped_features).eval()

print(mapped_features)
print(np.abs(qkernel_mat))

ListOp(
[CircuitStateFn(
     ┌───┐ ┌────────────┐                         ┌───┐ ┌────────────┐      »
q_0: ┤ H ├─┤ U1(1.0721) ├───■──────────────────■──┤ H ├─┤ U1(1.0721) ├───■──»
     ├───┤┌┴────────────┴┐┌─┴─┐┌────────────┐┌─┴─┐├───┤┌┴────────────┴┐┌─┴─┐»
q_1: ┤ H ├┤ U1(0.087602) ├┤ X ├┤ U1(16.143) ├┤ X ├┤ H ├┤ U1(0.087602) ├┤ X ├»
     └───┘└──────────────┘└───┘└────────────┘└───┘└───┘└──────────────┘└───┘»
«                        
«q_0: ────────────────■──
«     ┌────────────┐┌─┴─┐
«q_1: ┤ U1(16.143) ├┤ X ├
«     └────────────┘└───┘
),
CircuitStateFn(
     ┌───┐┌─────────────┐                        ┌───┐┌─────────────┐     »
q_0: ┤ H ├┤ U1(0.23478) ├──■──────────────────■──┤ H ├┤ U1(0.23478) ├──■──»
     ├───┤└┬────────────┤┌─┴─┐┌────────────┐┌─┴─┐├───┤└┬────────────┤┌─┴─┐»
q_1: ┤ H ├─┤ U1(1.2346) ├┤ X ├┤ U1(15.268) ├┤ X ├┤ H ├─┤ U1(1.2346) ├┤ X ├»
     └───┘ └────────────┘└───┘└────────────┘└───┘└───┘ └────────────┘└───┘»
«                        
«q_0: ────────────────■──
«   

## Converters

Converters traverse an Operator structure and perform a particular manipulation or replacement, defined by the converter's `convert()` method, of the Operators within. Typically, if a converter encounters an `OperatorBase` in the recursion which is irrelevant to its conversion purpose, that `OperatorBase` is left alone. Let's go back to our example from the top of this notebook to see the core converters in action.

In [23]:
import numpy as np
from qiskit.aqua.operators import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution
from qiskit.circuit import Parameter
from qiskit import BasicAer

### Evolutions, `exp_i()` and the `EvolvedOp`

Every `PrimitiveOp` and `ListOp` has an `.exp_i()` function to perform its Schrödinger-style exponentiation, $e^{-iHt}$. In practice, only a few of these Operators have an efficiently computable exponentiation (such as MatrixOp and the PauliOps with only one non-identity single-qubit Pauli), so we need to return a placeholder (similar to how `SummedOp` is a placeholder when we can't perform addition). This placeholder is called `EvolvedOp`, and it holds the `OperatorBase` to be exponentiated in its `.primitive` property.

Operator flow fully supports parameterization, so we can use a `Parameter` for our evolution time here. Notice that there's no "evolution time" argument in any function. The Operator flow exponentiates whatever operator we tell it to, and if we choose to multiply the operator by an evolution time, $e^{-iHt}$, that will be reflected in our exponentiation parameters. This is not some trick to make it look like Physics - it actually works this way under the hood.

In [24]:
two_qubit_H2 =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)

bell = CX @ (H^I) @ Zero
# We can also do CX @ (Plus ^ Zero)
evo_time = Parameter('θ')

evolution_op = (evo_time*two_qubit_H2).exp_i()
print(evolution_op) # Note, EvolvedOps print as exponentiations
print(repr(evolution_op))

e^(-i*1.0*θ * SummedOp(
[-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX]))
EvolvedOp(SummedOp([PauliOp(Pauli(z=[False, False], x=[False, False]), coeff=-1.0523732), PauliOp(Pauli(z=[True, False], x=[False, False]), coeff=0.39793742), PauliOp(Pauli(z=[False, True], x=[False, False]), coeff=-0.3979374), PauliOp(Pauli(z=[True, True], x=[False, False]), coeff=-0.0112801), PauliOp(Pauli(z=[False, False], x=[True, True]), coeff=0.18093119)], coeff=1.0*θ, abelian=False), coeff=1.0)


Now construct the energy measurement, (as discussed in [the `OperatorStateFn` section](#OperatorStateFn) above), and compose

In [25]:
h2_measurement = StateFn(two_qubit_H2).adjoint()
evo_and_meas = h2_measurement @ evolution_op @ bell
print(evo_and_meas)

ComposedOp(
[OperatorMeasurement(SummedOp(
[-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX])),
e^(-i*1.0*θ * SummedOp(
[-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX])),
CircuitStateFn(
               
q_0: ───────■──
     ┌───┐┌─┴─┐
q_1: ┤ H ├┤ X ├
     └───┘└───┘
)])


`Evolution`s are converters which traverse an Operator tree, replacing any `EvolvedOp` with a Schrödinger equation-style evolution `CircuitOp` equaling or approximating the matrix exponential of -i * the `OperatorBase` contained inside the `EvolvedOp` (`evo_op.primitive`). The Evolutions are essentially implementations of Hamiltonian Simulation algorithms, including various methods for Trotterization.

We can use an `Evolution` converter to convert `EvolvedOp`s into `CircuitOp`s. The `PauliTrotterEvolution` will convert the Operator to be evolved into a sum of `PauliOp`s and use Trotterization to produce a circuit, and `MatrixEvolution` will convert to a `MatrixOp` and produce a circuit using Terra's `HamiltonianGate` (which is classical exponentiation under the hood). 

Note that below, evolution time parameter θ has flowed into the circuit.

In [26]:
trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evo_and_meas)
# We can also set trotter_mode='suzuki' or leave it empty to default to first order Trotterization.
print(trotterized_op)

ComposedOp(
[OperatorMeasurement(SummedOp(
[-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX])),
CircuitStateFn(
               ┌───┐┌───┐┌───────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090465595*θ) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└───────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■─────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘                               └───┘     »
«     ┌───────────────────┐┌───┐┌──────────────────┐┌──────────────────┐┌───┐»
«q_0: ┤ RZ(-0.00564005*θ) ├┤ X ├┤ RZ(0.19896871*θ) ├┤ RZ(0.19896871*θ) ├┤ X ├»
«     └───────────────────┘└─┬─┘├──────────────────┤├──────────────────┤└─┬─┘»
«q_1: ───────────────────────■──┤ RZ(-0.1989687*θ) ├┤ RZ(-0.1989687*θ) ├──■──»
«                               └──────────────────┘└──────────────────┘     »
«     ┌───────────────────┐┌───┐┌───┐┌───┐┌───────────────────┐┌───┐┌───┐
«q_0: ┤ RZ(-0.00564005*θ) ├┤ X ├┤ H ├┤ X ├┤ RZ(0.090465595*θ) ├┤ X ├

We can bind our parameter to the operator if we so choose, and it will recursively bind into the circuit.

In [27]:
bound = trotterized_op.bind_parameters({evo_time: .5})
# Note bound is a `ComposedOp`, so here we are indexing [1] to just view the `CircuitOp`
bound[1].to_circuit().draw()

┌───┐┌───┐┌──────────────┐┌───┐┌───┐┌───┐┌──────────────┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.045233) ├┤ X ├┤ H ├┤ X ├┤ RZ(-0.00282) ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────┘└─┬─┘├───┤└─┬─┘└──────────────┘└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────■──┤ H ├──■────────────────────■──»
     └───┘└───┘└───┘                          └───┘                          »
«      ┌──────────────┐ ┌──────────────┐┌───┐┌──────────────┐┌───┐┌───┐┌───┐»
«q_0: ─┤ RZ(0.099484) ├─┤ RZ(0.099484) ├┤ X ├┤ RZ(-0.00282) ├┤ X ├┤ H ├┤ X ├»
«     ┌┴──────────────┤┌┴──────────────┤└─┬─┘└──────────────┘└─┬─┘├───┤└─┬─┘»
«q_1: ┤ RZ(-0.099484) ├┤ RZ(-0.099484) ├──■────────────────────■──┤ H ├──■──»
«     └───────────────┘└───────────────┘                          └───┘     »
«     ┌──────────────┐┌───┐┌───┐
«q_0: ┤ RZ(0.045233) ├┤ X ├┤ H ├
«     └──────────────┘└─┬─┘├───┤
«q_1: ──────────────────■──┤ H ├
«                          └───┘

### Expectations

`Expectation`s are converters which enable the computation of expectation values of Observables. They traverse an Operator tree, replacing `OperatorStateFn` measurements with equivalent measurements which are more amenable to
computation on quantum or classical hardware. For example, if we want to measure the expectation value of an Operator `o` expressed as a sum of Paulis with respect to some state function, but can only access diagonal measurements on Quantum hardware, we can create a measurement ~StateFn(o), use a ``PauliExpectation`` to convert it to a diagonal measurement and circuit pre-rotations to append to the state.

Another interesting `Expectation` is the `AerPauliExpectation`, which converts the measurement into a `CircuitStateFn` containing a special expectation snapshot instruction which `Aer` can execute natively with high performance.

In [28]:
# Note that XX was the only non-diagonal measurement in our H2 Observable
print(PauliExpectation(group_paulis=False).convert(h2_measurement))

SummedOp(
[ComposedOp(
[OperatorMeasurement(-1.0523732 * II),
II]),
ComposedOp(
[OperatorMeasurement(0.39793742 * IZ),
II]),
ComposedOp(
[OperatorMeasurement(-0.3979374 * ZI),
II]),
ComposedOp(
[OperatorMeasurement(-0.0112801 * ZZ),
II]),
ComposedOp(
[OperatorMeasurement(0.18093119 * ZZ),
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘])])


By default `group_paulis=True`, which will use the `AbelianGrouper` to convert the `SummedOp` into groups of mutually-commuting Paulis. This reduced circuit execution overhead, as each group can share the same circuit execution.

In [29]:
print(PauliExpectation().convert(h2_measurement))

SummedOp(
[ComposedOp(
[OperatorMeasurement(AbelianSummedOp(
[0.18093119 * ZZ,
-1.0523732 * II])),
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘]),
ComposedOp(
[OperatorMeasurement(AbelianSummedOp(
[0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ])),
II])])


Note that converters act recursively, so we can just convert our full evolution and measurement expression. We could have equally composed the converted `h2_measurement` with our evolution `CircuitStateFn`.

In [30]:
diagonalized_meas_op = PauliExpectation().convert(trotterized_op)
print(diagonalized_meas_op)

SummedOp(
[ComposedOp(
[OperatorMeasurement(AbelianSummedOp(
[0.18093119 * ZZ,
-1.0523732 * II])),
CircuitStateFn(
               ┌───┐┌───┐┌───────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090465595*θ) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└───────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■─────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘                               └───┘     »
«     ┌───────────────────┐┌───┐┌──────────────────┐┌──────────────────┐┌───┐»
«q_0: ┤ RZ(-0.00564005*θ) ├┤ X ├┤ RZ(0.19896871*θ) ├┤ RZ(0.19896871*θ) ├┤ X ├»
«     └───────────────────┘└─┬─┘├──────────────────┤├──────────────────┤└─┬─┘»
«q_1: ───────────────────────■──┤ RZ(-0.1989687*θ) ├┤ RZ(-0.1989687*θ) ├──■──»
«                               └──────────────────┘└──────────────────┘     »
«     ┌───────────────────┐┌───┐┌───┐┌───┐┌───────────────────┐┌───┐┌───┐┌───┐
«q_0: ┤ RZ(-0.00564005*θ) ├┤ X ├┤ H ├┤ X ├┤ RZ(0.090465595*θ) ├┤ X ├┤ H ├┤ H ├
«     └──────────

And now, we shard even further by binding with multiple parameters into a `ListOp`. Now that we've bound our parameters, we can already used `eval` (we could have used it earlier if we bound earlier), but it will not be efficient. It will convert our `CircuitStateFn`s to `VectorStateFn`s through simulation internally.

In [31]:
evo_time_points = list(range(8))
h2_trotter_expectations = diagonalized_meas_op.bind_parameters({evo_time: evo_time_points})

print(np.array(h2_trotter_expectations.eval()))

[-0.65443578+0.j -0.65411671+0.j -0.65077588+0.j -0.64558521+0.j
 -0.65582129+0.j -0.70897185+0.j -0.81391221+0.j -0.93426771+0.j]


### Executing `CircuitStateFn`s with the `CircuitSampler`

The `CircuitSampler` traverses an Operator and converts any `CircuitStateFns` into approximations of the state function by a `DictStateFn` or `VectorStateFn` using a quantum backend. Note that in order to approximate the value of the `CircuitStateFn`, it must 1) send state function through a depolarizing channel, which will destroy all phase information and 2) replace the sampled frequencies with **square roots** of the frequency, rather than the raw probability of sampling (which would be the equivalent of sampling the **square** of the state function, per the Born rule.

In [34]:
sampler = CircuitSampler(backend=BasicAer.get_backend('qasm_simulator'))
sampler.quantum_instance.run_config.shots = 1000
sampled_trotter_exp_op = sampler.convert(h2_trotter_expectations)
sampled_trotter_energies = sampled_trotter_exp_op.eval()
print('Sampled Trotterized energies:\n {}'.format(np.real(sampled_trotter_energies)))

Sampled Trotterized energies:
 [-0.66129419 -0.65874195 -0.65335021 -0.6339164  -0.67400049 -0.6923974
 -0.79415901 -0.94551284]


Note again that the circuits are replaced by dicts with ***squareroots*** of the circuit sampling probabilities. Take a look at one sub-expression before and after the conversion:

In [47]:
# print('Before:\n')
print(h2_trotter_expectations.reduce()[0][0])
print('\nAfter:\n')
print(sampled_trotter_exp_op[0][0])

ComposedOp(
[OperatorMeasurement(AbelianSummedOp(
[0.18093119 * ZZ,
-1.0523732 * II])),
CircuitStateFn(
               ┌───┐┌───┐┌───────┐┌───┐┌───┐┌───┐┌───────┐┌───┐┌───────┐»
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0) ├┤ X ├┤ H ├┤ X ├┤ RZ(0) ├┤ X ├┤ RZ(0) ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└───────┘└─┬─┘├───┤└─┬─┘└───────┘└─┬─┘├───────┤»
q_1: ┤ H ├┤ X ├┤ H ├──■─────────────■──┤ H ├──■─────────────■──┤ RZ(0) ├»
     └───┘└───┘└───┘                   └───┘                   └───────┘»
«     ┌───────┐┌───┐┌───────┐┌───┐┌───┐┌───┐┌───────┐┌───┐┌───┐┌───┐
«q_0: ┤ RZ(0) ├┤ X ├┤ RZ(0) ├┤ X ├┤ H ├┤ X ├┤ RZ(0) ├┤ X ├┤ H ├┤ H ├
«     ├───────┤└─┬─┘└───────┘└─┬─┘├───┤└─┬─┘└───────┘└─┬─┘├───┤├───┤
«q_1: ┤ RZ(0) ├──■─────────────■──┤ H ├──■─────────────■──┤ H ├┤ H ├
«     └───────┘                   └───┘                   └───┘└───┘
)])

After:

ComposedOp(
[OperatorMeasurement(AbelianSummedOp(
[0.18093119 * ZZ,
-1.0523732 * II])),
DictStateFn({'01': 0.726636084983398, '00': 0.6870225614927067})])


## Conclusion

The Operator flow grants you a flexible and fun set of building blocks to use for algorithm construction and prototyping. Take a look at the [Operator flow API reference](https://qiskit.org/documentation/apidoc/qiskit.aqua.operators.html#module-qiskit.aqua.operators) for more detailed information about methods and interfaces.